In [1]:
import rpwf
from rpwf import database, rpwf
from pathlib import Path
from typing import Dict
from dataclasses import dataclass
import pandas
from sklearn.model_selection import (
    RepeatedStratifiedKFold,
    GridSearchCV,
    cross_val_score
)

In [2]:
tmp_dir = Path("/tmp/RtmpzJiZbd/vignette1364d3efb3da5")
tmp_dir_posix = str(tmp_dir.as_posix())
print(tmp_dir_posix)

/tmp/RtmpzJiZbd/vignette1364d3efb3da5


In [3]:
os.listdir(tmp_dir)

['board.yml', 'db.SQLite']

In [4]:
@dataclass
class TestArgs:
    db_path: str
    board: str
    cores: int
    inner_n_cv: int
    inner_n_repeats: int
    outer_n_cv: int
    outer_n_repeats: int

In [9]:
args = TestArgs(tmp_dir.joinpath('db.SQLite'), tmp_dir.joinpath('board.yml'), 3, 2, 1, 2, 1)
wflow_id = 6

In [10]:
db_obj = database.Base(args.db_path)
board_obj = database.Board(args.board)
wflow_df = db_obj.all_wflow()

In [11]:
wflow_df

,wflow_id,model_tag,recipe_tag,result_path
0,1,XGBClassifier,xgb_base,None
1,2,XGBClassifier,xgb_pca,None
2,3,svm_rbf,scaled_base,None
3,4,glmnet,scaled_base,None
4,5,svm_rbf,scaled_pca,None
5,6,glmnet,scaled_pca,None


In [12]:
wflow_obj = rpwf.Wflow(db_obj, board_obj, wflow_id)
n_cores = args.cores

# Generate the parameters
p_grid = rpwf.RGrid(db_obj, board_obj, wflow_obj).get_grid()

df_obj = rpwf.TrainDf(db_obj, board_obj, wflow_obj)
X, y = df_obj.get_df_X(), df_obj.get_df_y()

model_type_obj = rpwf.Model(db_obj, board_obj, wflow_obj)
base_learner = rpwf.BaseLearner(wflow_obj, board_obj, model_type_obj).base_learner
score = wflow_obj._get_par("costs")

AttributeError: 'DataFrame' object has no attribute 'to_pylist'

In [ ]:
p_grid

In [ ]:
test = p_grid[1]

In [ ]:
test

In [ ]:
def val_to_list(d: Dict):
    for v in d:
        d[v] = [d[v]]
    return d

In [ ]:
wrapped_grid = [None] * len(p_grid)

In [ ]:
for i in range(len(p_grid)):
    wrapped_grid[i]= val_to_list(p_grid[i])

In [ ]:
wrapped_grid

In [ ]:
# Nested resampling
inner_cv = RepeatedStratifiedKFold(
    n_splits=args.inner_n_cv, 
    n_repeats=args.inner_n_repeats,
    random_state=wflow_obj.random_state
)
outer_cv = RepeatedStratifiedKFold(
    n_splits=args.outer_n_cv,
    n_repeats=args.outer_n_repeats,
    random_state=wflow_obj.random_state,
)

if p_grid is None:
    print("No tune grid specified, running with default params")
    nested_score = cross_val_score(
        base_learner, X=X, y=y, cv=outer_cv, n_jobs=n_cores, scoring=score
    )

else:
    print("Performing nested-cv using provided Rgrid")
    param_tuner = GridSearchCV(
        estimator=base_learner,
        param_grid=p_grid,
        cv=inner_cv,
        n_jobs=n_cores,
        scoring=score,
    )
    nested_score = cross_val_score(param_tuner, X=X, y=y, cv=outer_cv)

In [ ]:
# if args.export:
    # Export the results
exporter = rpwf.Export(db_obj, wflow_obj)
nested_score_df = pandas.DataFrame(nested_score, columns=[score])
exporter.export_cv(nested_score_df, "nested_cv")
exporter.export_db()

In [ ]:
%run -m rpwf.script.nested_resampling -h

In [ ]:
%run -m rpwf.script.nested_resampling $tmp_dir_posix -db db.SQLite -s

In [ ]:
%run -m rpwf.script.nested_resampling $tmp_dir_posix -db db.SQLite -af -c 7 -icv 5 -icr 1 -ocv 5 -ocr 1